In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.Load('bangla_char.model')


In [ ]:
text = "আমার বাংলা ভাষা আমার গর্ব"
tokens = sp.encode_as_ids(text)
print(tokens)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import wandb

# Login to Weights & Biases
wandb.login(key='WANDBKEY')


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


In [ ]:
import wandb
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import sentencepiece as spm
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import DataCollatorForLanguageModeling
from custom_tokenizer import CustomBengaliTokenizer 

wandb.init(project="Transtokenizers")

# custom tokenizer
tokenizer = CustomBengaliTokenizer(vocab_file="bangla_char.vocab", model_file="bangla_char.model")

def tokenize_text_file(file_path, tokenizer):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    tokenized_lines = [tokenizer.encode(line.strip()) for line in lines if line.strip()]
    return tokenized_lines

file_path = 'final_cleaned_bangla_corpus.txt'
tokenized_data = tokenize_text_file(file_path, tokenizer)

class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.tokenized_data[idx], dtype=torch.long)
        return {'input_ids': input_ids, 'labels': input_ids.clone()}

dataset = CustomDataset(tokenized_data)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4) 

model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B`")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2,
    gradient_accumulation_steps=4, 
    evaluation_strategy="steps",
    eval_steps=1000,
    fp16=True,
    dataloader_num_workers=4,
    report_to="wandb",
    run_name="Transtokenizers_run"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

trainer.train()

wandb.finish()


In [ ]:


from unsloth import FastLanguageModel
import torch
from datasets import Dataset
import sentencepiece as spm
from tqdm import tqdm
import wandb


wandb.init(project="Transtokenizers")


def load_and_tokenize_text(file_path, sp_model):
    sp = spm.SentencePieceProcessor(model_file=sp_model)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    tokenized_lines = [sp.encode(line.strip(), out_type=int) for line in tqdm(lines, desc="Tokenizing lines") if line.strip()]
    return tokenized_lines


file_path = 'final_cleaned_bangla_corpus.txt'
sp_model = 'bangla_bpe.model'
tokenized_data = load_and_tokenize_text(file_path, sp_model)


def create_hf_dataset(tokenized_data):
    texts = [" ".join(map(str, tokens)) for tokens in tokenized_data]
    dataset = Dataset.from_dict({"text": texts})
    return dataset

dataset = create_hf_dataset(tokenized_data)


model_name = "unsloth/Meta-Llama-3.1-8B"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,  
    load_in_4bit=True,  
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

def format_data(examples):
    texts = examples["text"]
    formatted_texts = [text + tokenizer.eos_token for text in texts]
    return {"text": formatted_texts}

dataset = dataset.map(format_data, batched=True, desc="Formatting data")

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="wandb",
        run_name="Transtokenizers_run"
    ),
)


trainer_stats = trainer.train()


gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"Training runtime: {trainer_stats.metrics['train_runtime']} seconds")
print(f"Training runtime in minutes: {round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes")
print(f"Peak reserved memory: {used_memory} GB")
print(f"Peak reserved memory for training: {used_memory_for_lora} GB")
print(f"Peak reserved memory % of max memory: {used_percentage} %")
print(f"Peak reserved memory for training % of max memory: {lora_percentage} %")


model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")


wandb.finish()


In [ ]:

import requests

def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()  
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

ben_url = 'https://raw.githubusercontent.com/BibleNLP/ebible/main/corpus/ben-ben2017.txt'
eng_url = 'https://raw.githubusercontent.com/BibleNLP/ebible/main/corpus/eng-enggnv.txt'


download_file(ben_url, 'ben-ben2017.txt')
download_file(eng_url, 'eng-enggnv.txt')



import sentencepiece as spm


corpus_file = 'eng-enggnv.txt'
model_prefix = 'english_bpe'
vocab_size = 32000

spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix='english_bpe',
    vocab_size=32000,
    model_type='bpe',
    character_coverage=0.9995,
    max_sentence_length=10000
)

print(f"Tokenizer trained and saved as {model_prefix}.model and {model_prefix}.vocab")





In [8]:
import sentencepiece as spm

# Define paths and parameters for English tokenizer
corpus_file_en = 'eng-enggnv.txt'  # Path to your English corpus
model_prefix_en = 'english_bpe'  # Prefix for the output model files
vocab_size_en = 32000  # Set your desired vocabulary size

# Train the English tokenizer
spm.SentencePieceTrainer.train(
    input=corpus_file_en,
    model_prefix=model_prefix_en,
    vocab_size=vocab_size_en,
    model_type='bpe',
    character_coverage=0.9995,  
    max_sentence_length=10000
)

print(f"English tokenizer trained and saved as {model_prefix_en}.model and {model_prefix_en}.vocab")


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: eng-enggnv.txt
  input_format: 
  model_prefix: english_bpe
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 10000
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  di

KeyboardInterrupt: 

In [1]:
# preparing paraellel corpus
def prepare_parallel_corpus(eng_file, ben_file, output_file):
    with open(eng_file, 'r', encoding='utf-8') as eng, open(ben_file, 'r', encoding='utf-8') as ben, open(output_file, 'w', encoding='utf-8') as out:
        eng_lines = eng.readlines()
        ben_lines = ben.readlines()
        
      
        assert len(eng_lines) == len(ben_lines), "The English and Bengali files must have the same number of lines."
        
        for eng_line, ben_line in zip(eng_lines, ben_lines):
            eng_line = eng_line.strip()
            ben_line = ben_line.strip()
            if eng_line and ben_line: 
                out.write(f"{eng_line} ||| {ben_line}\n")

prepare_parallel_corpus('eng-enggnv.txt', 'ben-ben2017.txt', 'parallel_corpus.txt')


In [5]:
#  Aligning the Parallel Corpus Using FastAlign:
import os

fast_align_dir = 'fast_align/build'  

os.system(f'chmod +x {fast_align_dir}/fast_align')
os.system(f'chmod +x {fast_align_dir}/atools')


0

In [6]:
#  Aligning the Parallel Corpus Using FastAlign:
!{fast_align_dir}/fast_align -i parallel_corpus.txt -d -o -v > forward.align

!{fast_align_dir}/fast_align -i parallel_corpus.txt -d -o -v -r > reverse.align

!{fast_align_dir}/atools -i forward.align -j reverse.align -c grow-diag-final-and > bidirectional.align


ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
..............................
expected target length = source length * 0.827891
ITERATION 1
..............................
  log_e likelihood: -1.3105e+07
  log_2 likelihood: -1.89066e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.167949
       size counts: 1679
ITERATION 2
..............................
  log_e likelihood: -4.77458e+06
  log_2 likelihood: -6.88827e+06
     cross entropy: 10.8926
        perplexity: 1901.01
      posterior p0: 0.0820574
 posterior al-feat: -0.138536
       size counts: 1679
  1  model al-feat: -0.213993 (tension=4)
  2  model al-feat: -0.17441 (tension=5.50914)
  3  model al-feat: -0.160126 (tension=6.22663)
  4  model al-feat: -0.152734 (tension=6.65843)
  5  model al-feat: -0.14834 (tension=6.9424)
  6  model al-feat: -0.145519 (tension=7.13849)
  7  model al-feat: -0.143616 (tension=7.27816)
  8  model al-feat: -0.142287 (tension=7.37976)
     final ten

In [7]:
def read_alignment_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        alignments = file.readlines()
    return alignments

forward_alignments = read_alignment_file('forward.align')
reverse_alignments = read_alignment_file('reverse.align')
bidirectional_alignments = read_alignment_file('bidirectional.align')


print("Forward Alignments ")
print(forward_alignments[:5])

print("\nReverse Alignments ")
print(reverse_alignments[:5])

print("\nBidirectional Alignments:")
print(bidirectional_alignments[:5])


Forward Alignments 
['2-0 3-1 4-2 5-3 6-4 4-5 9-6\n', '2-0 2-1 4-2 6-3 5-4 10-5 8-6 9-7 10-8 11-9 10-10 14-11 16-12 16-13 22-14 20-15 22-16 22-17\n', '0-0 1-1 2-2 6-3 6-4 7-5 10-6 10-7\n', '0-0 1-1 4-2 8-3 2-4 9-5 10-6 13-7 12-8 13-9 11-10 16-11\n', '0-0 1-1 4-2 2-3 5-4 6-5 8-6 10-7 11-8 10-9 14-10 15-11 17-12 17-13 20-14 21-15 21-16\n']

Reverse Alignments 
['0-0 1-0 2-0 3-1 4-5 5-4 6-2 7-3 8-5 9-4\n', '0-11 1-0 2-0 3-5 4-2 5-4 6-3 7-4 8-6 9-7 10-5 11-9 12-9 13-14 14-6 15-13 16-13 17-12 18-12 19-16 20-15 21-15 22-14\n', '0-0 1-1 2-2 3-4 4-3 5-4 6-3 7-5 8-6 9-7 10-6\n', '0-0 1-1 2-4 3-2 4-2 5-4 6-4 7-4 8-3 9-5 10-6 11-10 12-8 13-7 14-10 15-10 16-9\n', '0-0 1-1 2-3 3-2 4-4 5-4 6-5 7-5 8-6 10-7 11-8 12-10 13-9 14-10 15-11 16-12 17-12 18-16 19-14 20-14 21-16\n']

Bidirectional Alignments:
['0-0 1-0 2-0 3-1 4-2 4-5 5-3 5-4 6-2 7-3 9-6\n', '1-0 2-0 2-1 4-2 5-4 6-3 7-4 8-6 9-7 10-5 10-8 10-10 11-9 12-9 14-11 15-13 16-12 16-13 17-12 18-12 19-16 20-15 21-15 22-14\n', '0-0 1-1 2-2 3-4 4-3 5-4 6

In [ ]:
from collections import defaultdict
import sentencepiece as spm


def read_alignment_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        alignments = file.readlines()
    return alignments

forward_alignments = read_alignment_file('forward.align')
reverse_alignments = read_alignment_file('reverse.align')
bidirectional_alignments = read_alignment_file('bidirectional.align')

def create_token_mapping(alignments):
    token_map = defaultdict(list)
    for line in alignments:
        parts = line.strip().split()
        for part in parts:
            en_idx, bn_idx = part.split('-')
            token_map[int(en_idx)].append(int(bn_idx))
    return token_map


token_map = create_token_mapping(bidirectional_alignments)

print(token_map)


In [ ]:
#failing to create smooth token mapping